In [ ]:
from cling import cling, bash

**Listing 13.1**

Caption: Implementation of the fractal set using the Message Passing Interface 

In [ ]:
%%writefile mpi_fractal.cpp
#include <mpi.h>
#include <chrono>

#include <config.hpp>
#include <pbm.hpp>
#include <kernel.hpp>
#include <util.hpp>
#include <io.hpp>
//-----------------------------------------------------------------

using std::chrono::high_resolution_clock;
int main() {
  // Initialize the MPI environment
  MPI_Init(NULL, NULL); 
  // Get the number of processes or nodes
  int mpi_size;
  MPI_Comm_size(MPI_COMM_WORLD, &mpi_size); 
  // Get the rank of the process
  int mpi_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &mpi_rank); 
  // Start the timer
  high_resolution_clock::time_point start_time;
  PBM pbm;
  if (mpi_rank == 0){
    start_time  = high_resolution_clock::now(); 
  }
  pbm = PBM(size_y, size_x);

  std::size_t iter_start, iter_end;
  split_work(mpi_size, size_x, mpi_rank, iter_start, iter_end); 

  for (size_t iter = iter_start; iter < iter_end; iter++) { 
    pbm.row(iter) = compute_row(iter);
    assert(pbm.row(iter).size() == size_y);
    if(mpi_rank > 0) {
       MPI_Request req;
       MPI_Isend((void*)pbm.row(iter).data(),size_y,MPI_INT,0,iter,MPI_COMM_WORLD,&req); 
    }
  } 

  // Save the image
  if (mpi_rank == 0) { 
     save_image(pbm, mpi_size, size_x, size_y);
  }
  // Stop the timer
  if (mpi_rank == 0){
    auto stop_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(
        stop_time - start_time);
    std::cout << "Time: " << duration.count()*1e-9 << std::endl;
  }
  // Finalize the MPI environment.
  MPI_Finalize();  
}


In [ ]:
%%bash
mpicxx -I . -o mpi_fractal.exe mpi_fractal.cpp
mpirun --bind-to none -np 3 ./mpi_fractal.exe

**Listing 13.2**

Caption: Utility functions to save the PBM image

In [ ]:
%%writefile io.hpp
#ifndef IO_HPP
#define IO_HPP

void save_image(PBM &pbm, size_t mpi_size, size_t size_x,
                size_t size_y) {
  std :: size_t iter_start , iter_end ;
  for(int worker=1;worker<mpi_size;worker++) { 
    split_work(mpi_size, size_x, worker, iter_start, iter_end);
    for(size_t iter=iter_start;iter < iter_end; ++iter) {  
      std::vector<int> v; 
      v.resize(size_y); 
      MPI_Status status;
      MPI_Recv((void*)v.data(),size_y,MPI_INT,worker,iter,MPI_COMM_WORLD,&status);   
      assert(pbm.row(iter).size() == v.size());
      pbm.row(iter) = v; 
    }
  }
  // Save the image
  pbm.save("image_mpi.pbm"); 
}
#endif


**Listing 13.3**

Caption: Implementation of the fractal set using the Message Passing Interface with distributed IO.

In [ ]:
%%writefile mpi_fractal_improved.cpp
#include <mpi.h>
#include <chrono>
#include <fstream> 

#include <config.hpp>
#include <kernel.hpp>
#include <util.hpp>

using std::chrono::high_resolution_clock;
//-----------------------------------------------------------------
int main() {
  // Initialize the MPI environment
  MPI_Init(NULL, NULL);
  // Get the number of processes or nodes
  int mpi_size;
  MPI_Comm_size(MPI_COMM_WORLD, &mpi_size);
  // Get the rank of the process
  int mpi_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &mpi_rank);
  // Start the timer
  high_resolution_clock::time_point start_time;
  std::ofstream outfile("data_"+
    std::to_string(mpi_rank)+".part"); 
  if (mpi_rank == 0)
    start_time  = high_resolution_clock::now();

  std::size_t iter_start, iter_end;
  split_work(mpi_size, size_x, mpi_rank, iter_start, iter_end);

  for (size_t iter = iter_start; iter < iter_end; iter++) {
    std::vector<int> row = compute_row(iter);

    assert(row.size() == size_y);
    outfile << iter << " ";
    for(auto color : row) 
      outfile << color << " ";
      outfile << "\n";
  }

  // Stop the timer
  if (mpi_rank == 0){
    auto stop_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(
        stop_time - start_time);
    std::cout << "Time: " << duration.count()*1e-9 << std::endl;
  }
  outfile.close(); 
  // Finalize the MPI environment.
  MPI_Finalize();
}


In [ ]:
%%bash
mpicxx -I . -o mpi_fractal_improved.exe mpi_fractal_improved.cpp
mpirun --bind-to none -np 3 ./mpi_fractal_improved.exe

**Listing 13.4**

Caption: Code to combine the distributed image data into a single PBM image.

In [ ]:
%%writefile combine_image.cpp
#include <fstream> 
#include <iostream>
#include <pbm.hpp> 
//-----------------------------------------------------------------
std::tuple<int, std::vector<int>> split(std::string line) { 

  std::istringstream stream(line); 
  std::string index;
  std::vector<int> row;

  std::string s;
  bool first = true;
  while (getline(stream, s, ' ')) { 
    if (first) {
      index = s;
      first = false;
    } else
      row.push_back(std::stoi(s));
  }
  return std::make_tuple(std::stoi(index), row);
}

int main(int argc, char **argv) {

  PBM pbm = PBM(size_y, size_x); 

  size_t id = 0;

  while (true) {  

    std::ifstream file; 
    file.open("data_" + std::to_string(id) + ".part");
    if(!file.good()) {
        std::cout << "Files read: " << id << std::endl;
        break;
    }

    int index; 
    std::vector<int> row; 

    std::string s;
    while (std::getline(file, s)) { 

      auto res = split(s);  

      index = std::get<0>(res); 
      row = std::get<1>(res); 
      pbm.row(index) = row; 
    }
    id++;
  }

  pbm.save("image_combined.pbm"); 

  return 0;
}


In [ ]:
%%bash
g++ -std=c++20 -I . -o combine_image.exe combine_image.cpp -lpthread
./combine_image.exe

**Listing 13.5**

Caption: Implementation of the fractal set using MPI + OpenMP: Using a critical section for the distributed I/O.

In [ ]:
%%cling
#pragma omp parallel for 
for (size_t iter = iter_start; iter < iter_end; iter++) {
  std::vector<int> row = compute_row(iter);

  assert(row.size() == size_y);
  #pragma omp critical 
  {
    outfile << iter << " ";
    for(auto color : row) {
      outfile << color << " ";
    }
    outfile << "\n";
  } 
}


**Listing 13.6**

Caption: Improved implementation of fractal set using MPI+OpenMP without using a scope lock.

In [ ]:
%%writefile mpi_openmp_fractal.cpp
#include <mpi.h>
#include <chrono>
#include <fstream>
#include <config.hpp>
#include <kernel.hpp>
#include <util.hpp>

using std::chrono::high_resolution_clock;
//-----------------------------------------------------------------
int main() {
  // Initialize the MPI environment
  MPI_Init(NULL, NULL);
  // Get the number of processes or nodes
  int mpi_size;
  MPI_Comm_size(MPI_COMM_WORLD, &mpi_size);
  // Get the rank of the process
  int mpi_rank;
  MPI_Comm_rank(MPI_COMM_WORLD, &mpi_rank);
  // Start the timer
  high_resolution_clock::time_point start_time;
  std::ofstream outfile ("data_"+std::to_string(mpi_rank)+".part");
  if (mpi_rank == 0)
    start_time  = high_resolution_clock::now();

  std::size_t iter_start, iter_end;
  split_work(mpi_size, size_x, mpi_rank, iter_start, iter_end); 

  std::vector<std::vector<int>> data(iter_end-iter_start); 

  #pragma omp parallel for   
  for (size_t iter = iter_start; iter < iter_end; iter++) { 
    data[iter-iter_start] = compute_row(iter);
  }

  size_t iter  = iter_start;  
  for( auto row : data){
     outfile << iter << " ";
     for(auto color : row)
       outfile << color << " ";
     outfile << "\n";
     iter++;
  }  

  // Stop the timer
  if (mpi_rank == 0){
    auto stop_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::nanoseconds>(
    stop_time - start_time);
    std::cout << "Time: " << duration.count()*1e-9 << std::endl;
  }
  outfile.close();
  // Finalize the MPI environment.
  MPI_Finalize();
}


In [ ]:
%%bash
mpicxx -I . -o mpi_openmp_fractal.exe mpi_openmp_fractal.cpp
mpirun --bind-to none -np 3 ./mpi_openmp_fractal.exe

**Listing 13.7**

Caption: Sample bash script to to run the distributed MPI-based fractal set code.

In [ ]:
%%bash
#!/bin/bash
# Figure out the number of cores
export CORES=$(lscpu|grep "^CPU(s):"|cut -d: -f2|sed 's/\s//g')
echo "CORES=${CORES}"
# Let OpenMP use half the machine
export OMP_NUM_THREADS=$((${CORES}/2))
echo "OMP_NUM_THREADS=${OMP_NUM_THREADS}" 
export SIZE_X=$((${CORES}*100)) 
export SIZE_Y=$((${CORES}*100))
echo "SIZE_X=${SIZE_X} SIZE_Y=${SIZE_Y}"
export TYPE=julia 
# Run in two processes
mpirun --bind-to none -N 2 ./mpi_fractal.exe 
